# 実験3 100行コンパイラ

In [ ]:
require("./cumps");
function print(s) { console.log(s); }

## 式のコンパイル

式の文法
<img src="pics/rail_expr.png">
(注:最後のexpr2は（もちろん）expr3の間違い)

<img src="rail_expr.png">

式のパーサの定義（上記の文法の直訳）

In [ ]:
function expr(s) { return expr(s); };
var id   = pat("[a-zA-Z_][a-zA-Z0-9_]*");
var num    = pat("[0-9]+");
var expr0  = oneOf(id, num, seq("(", expr, ")"));
var expr1  = sepBy(expr0, oneOf(w("*"), w("/")));
var expr2  = sepBy(expr1, oneOf(w("+"), w("-")));
var expr3  = sepBy(expr2, oneOf(w(">"), w("<")));
var expr   = sepBy(expr3, w("=="));

数（num）の翻訳（常にその数を返すような恒等関数に翻訳）

In [ ]:
num.action = function (s) {
    var n = parseInt(s);
    return function () { return n; };
};

テスト

In [ ]:
var f = num("123").a;
f()

識別子（変数）の翻訳（その変数の現在の値を返す関数に翻訳　変数の値はstateという名前のオブジェクト（連想配列）に保存されていると仮定）

In [ ]:
id.action = function (x) {
    return function () { return state[x]; };
};

テスト

In [ ]:
var f = id("x").a; 

In [ ]:
var state = { x : 777 }; // xの現在値を変えて再実行してみよ
f()

式（expr,expr1,expr2,expr3）の翻訳（前回の実験のEvalExprに酷似 電卓式の計算を実行する関数に翻訳）

In [ ]:
function compileExpr(ts) {
    return function() {
        var v = ts[0]();
        for (var i = 1; i < ts.length - 1; i += 2) {
            switch (ts[i]) {
              case "+": v += ts[i + 1](); break;
              case "-": v -= ts[i + 1](); break;
              case "*": v *= ts[i + 1](); break;
              case "/": v = parseInt(v / ts[i + 1]()); break;
              case "<": v = v < ts[i + 1](); break; 
              case ">": v = v > ts[i + 1](); break;
              case "==": v = v === ts[i + 1](); 
            }
        }
        return v;
    };
}

expr1.action = expr2.action = expr3.action = expr.action = compileExpr;

テスト

In [ ]:
var f = expr("x + y * 2").a;

In [ ]:
var state = { x : 1, y : 2 }; // x,yの現在値を変えて再実行してみよ
f()

[課題4] 式の文法を拡張して，以下の演算子をサポートせよ．

     %, <=, >=, !=
ただし，%の優先順位は*と同じであり、<=と>=の優先順位は<と同じであり，!=の優先順位は==と同じであるとする．     

## 実行文（ステートメント）のコンパイル

実行文の文法
<img src="pics/rail_stmt.png">

<img src="rail_stmt.png">

実行文のパーサの定義（上記の文法の直訳）

In [ ]:
function stmt(s) { return stmt(s); }
var lval       = pat("[a-zA-Z_][a-zA-Z0-9_]*");
var assignStmt = seq(lval, "=", expr, ";");
var whileStmt  = seq("while", "(", expr, ")", stmt);
var ifStmt     = seq("if", "(", expr, ")", stmt, opt(seq("else", stmt)));
var printStmt  = seq("print", expr, ";");
var program    = moreThan0(stmt);
var stmt       = oneOf(assignStmt, whileStmt, ifStmt, printStmt, seq("{", program, "}"));

代入文（assignStmt）の翻訳（stateのxの値を更新する関数に翻訳）

In [ ]:
assignStmt.action = function(ts) {
    var x = ts[0], evalRHS = ts[1];
    return function () { state[x] = evalRHS(); };
};

テスト

In [ ]:
var aout = assignStmt("x=x+1;").a;

In [ ]:
var state = { x : 0 };

In [ ]:
aout()
state

print文（printStmt）の翻訳（まず式の値を求めそれをprint関数を用いて表示する関数に翻訳）

In [ ]:
printStmt.action = function(ts) {
    var eval = ts[0];
    return function () { print(eval()); };
};

テスト

In [ ]:
var aout = printStmt("print x + y;").a;

In [ ]:
var state = {x : 1, y : 2};
aout()

複文（program）の翻訳（前から順に実行する関数に翻訳）

In [ ]:
program.action = function (ts) {
    return function() { for (i = 0; i < ts.length; i++) ts[i](); }; 
};

テスト

In [ ]:
var aout = program("x = 1; print x; x = x + 1; print x;").a;

In [ ]:
var state = {};
aout()

while文（whileStmt）の翻訳（while文の動作をJavaScriptのwhile文で実現するので自明）

In [ ]:
whileStmt.action = function(ts) {
    var evalC = ts[0], runStmt = ts[1];
    return function () { while (evalC()) runStmt(); };
};

テスト

In [ ]:
var aout = whileStmt("while (x > 0) { print x * x; x = x - 1; }").a;

In [ ]:
var state = {x : 10};
aout()

if文（ifStmt）の翻訳（同様にJavaScriptのif文で実現すればよいので簡単　ただしelse部分は省略されている場合は何もしない関数を補っている）

In [ ]:
ifStmt.action = function(ts) {
    var evalC = ts[0], runIf = ts[1], runElse = ts[2] ? ts[2] : function() {};
    return function() { if (evalC()) runIf(); else runElse(); };       
};

テスト

In [ ]:
var aout = ifStmt("if (0 < 1) print 111; else print 222;").a;
aout()

In [ ]:
var aout = ifStmt("if (0 > 1) print 111; else print 222;").a;
aout()

全体のテスト（0,1,2,...,9を出力するプログラム例）

In [ ]:
var aout = program("x = 0; while (x < 10) { print x; x = x + 1; }").a;
var state = {};
aout()

以下はフィボナッチ数列の最初の30個を出力する（JavaScriptで文字列を複数行に分けて与えるときはバックスラッシュが必要）

In [ ]:
var fib ="\
n = 30;\
a = 0;\
b = 1;\
while (n > 0) {\
  print b;\
  c = b;\
  b = a + b;\
  a = c;\
  n = n - 1;\
}\
"
state = {};
(program(fib).a)();

ファイルからソース・プログラムを読み込んで実行できるようにする

In [ ]:
var fs = require("fs");
function run(file) {
    var src = fs.readFileSync(file, {encoding : "utf8"});
    state = {};
    var aout = program(src).a;
    aout();
}

In [ ]:
run("./fib.toy");

[課題5] C言語風のdo-while文をサポートするようにプログラムを拡張せよ（do-while文のパーサーとアクション関数をつくり追加せよ）．　ヒント：JavaScriptにもC言語風のdo-while文があるのでこれを用いるとよい．